In [2]:
from IPython.core.display import HTML

In [2]:
src = "http://labrosa.ee.columbia.edu/millionsong/sites/default/files/tutorial1.pdf"
HTML("<iframe src='%s' width=1000 height=1000></iframe>"%src)

In [3]:
"""
Tutorial for the Million Song Dataset

by Thierry Bertin-Mahieux (2011) Columbia University
   tb2332@columbia.edu
   Copyright 2011 T. Bertin-Mahieux, All Rights Reserved

This tutorial will walk you through a quick experiment
using the Million Song Dataset (MSD). We will actually be working
on the 10K songs subset for speed issues, but the code should
transpose seamlessly.

In this tutorial, we do simple metadata analysis. We look at
which artist has the most songs by iterating over the whole
dataset and using an SQLite database.

You need to have the MSD code downloaded from GITHUB.
See the MSD website for details:
http://labrosa.ee.columbia.edu/millionsong/

If you have any questions regarding the dataset or this tutorial,
please first take a look at the website. Send us an email
if you haven't found the answer.

Note: this tutorial is developed using Python 2.6
      on an Ubuntu machine. PDF created using 'pyreport'.
"""

# usual imports
import os
import sys
import time
import glob
import datetime
import sqlite3
import numpy as np
# get it at: http://numpy.scipy.org/

In [4]:
!pwd ; ls

/media/1ADF-0E69/msong
kaggle		   millionsongsubset_full.tar.gz  tutorial_1.ipynb
MillionSongSubset  MSongsDB			  uci


# SETTING LOCAL FILESPATHS

In [16]:
# CHANGE IT TO YOUR LOCAL CONFIGURATION
msd_subset_path='/media/1ADF-0E69/msong/' + 'MillionSongSubset'
msd_subset_data_path=os.path.join(msd_subset_path,'data')
msd_subset_addf_path=os.path.join(msd_subset_path,'AdditionalFiles')
assert os.path.isdir(msd_subset_path),'wrong path' # sanity check
# path to the Million Song Dataset code
# CHANGE IT TO YOUR LOCAL CONFIGURATION
msd_code_path='/media/1ADF-0E69/msong/' + 'MSongsDB'
assert os.path.isdir(msd_code_path),'wrong path' # sanity check
# we add some paths to python so we can import MSD code
# Ubuntu: you can change the environment variable PYTHONPATH
# in your .bashrc file so you do not have to type these lines
sys.path.append( os.path.join(msd_code_path,'PythonSrc') )

In [6]:
import hdf5_getters as GETTERS

# HELPER FUNCTIONS

In [7]:
# the following function gives us a nice string for time lag in seconds
def strtimedelta(starttime, stoptime):
    return str(datetime.timedelta(seconds=stoptime-starttime))

In [8]:
# we define a very useful function to iterate over files
def apply_to_all_files(basedir, func=lambda x : x, ext='.h5'):
    """ From a base dir, go through all subdirectories,
    find all the files with given extensions, apply the given function 'func' to all of them
    If no 'func' is passed, we do nothing except counting.
    
    RETURN:
    number of files
    """
    cnt = 0
    
    # iterate over all files in subdirectories 
    for root, dirs, files in os.walk(basedir):
        files = glob.glob(os.path.join(root, "*" + ext))
        # count files
        cnt += len(files)
        
        # apply function to all files
        for f in files:
            func(f)
            
    return cnt

# we can now easily count the number of files in the dataset
print 'number of song files: ', apply_to_all_files(msd_subset_data_path)

number of song files:  10000


# Artist Names

In [9]:
# let's get all artists name in a Python set collection
all_artists_name = set()

# we define the function to apply to all files
def func_to_get_artist_name(filename):
    """
    This function does 3 simple things:
    - open the file song
    - get artist ID and put it
    - close the file
    """
    h5 = GETTERS.open_h5_file_read(filename)
    artist_name = GETTERS.get_artist_name(h5)
    all_artists_name.add( artist_name )
    h5.close()

# let's apply the previous function to all the files
# we'll also measure how long it takes
t1 = time.time()
apply_to_all_files(msd_subset_data_path, func=func_to_get_artist_name)
t2 = time.time()

print 'all artists name extracted in:', strtimedelta(t1, t2)
print 'head of artist name set: ', list(all_artists_name)[:10]

print 'found', len(all_artists_name), 'unique artist names'

all artists name extracted in: 0:03:31.719018
head of artist name set:  ['Groundhogs', 'Pale Forest', 'The Real Kids', 'JennyAnyKind', 'Aswad', 'Little Willie John', 'Barry Goldberg', 'Spooky Tooth / Mike Harrison', 'Red Foley', '4 Skins']
found 4412 unique artist names


# Artist Names (USING SQLITE)

In [17]:
# this takes too long, and artist names has already been done
# we'll try conecting to a SQLite Database: track_metadata.db
conn = sqlite3.connect(os.path.join(msd_subset_addf_path,
                                   'subset_track_metadata.db'))

In [24]:
q = "SELECT DISTINCT artist_name FROM songs"
t1 = time.time()
res = conn.execute(q)
all_artist_names_sqlite = res.fetchall()
t2 = time.time()
print "all artist names extracted in SQLite in: ", strtimedelta(t1, t2)

all artist names extracted in SQLite in:  0:00:00.186274


In [25]:
conn.close()

In [28]:
for k in range(5):
    print all_artist_names_sqlite[k][0]

!!!
(hed) p.e.
089 Clique feat. Minnesota Snipe & Skinny Cueball
089 Clique feat. Prophet
1. Futurologischer Congress


# Artist IDs (USING SQLITE)

In [29]:
# now, let's find the artist with the most songs in the dataset
# work with artist ID, not artist names. Some artists have many names, usually because the song is
# featuring someone else
conn = sqlite3.connect(os.path.join(msd_subset_addf_path,
                                    'subset_track_metadata.db'))

# we build a sql query
q = "SELECT DISTINCT artist_id FROM songs"
res = conn.execute(q)
all_artists_ids = map(lambda x : x[0], res.fetchall())
conn.close()

for k in range(4):
    print all_artists_ids[k]

AR009211187B989185
AR00A6H1187FB5402A
AR00LNI1187FB444A5
AR00MBZ1187B9B5DB1


# COUNT ARTIST SONG COUNTS

In [31]:
## count number songs per artist
files_per_artist = {}
for aid in all_artists_ids:
    files_per_artist[aid] = 0
    
def func_to_count_artist_id(filename):
    """
    This function does 3 things
    - open the song file
    - get artist ID and put it
    - close the file
    """
    h5 = GETTERS.open_h5_file_read(filename)
    artist_id = GETTERS.get_artist_id(h5)
    files_per_artist[artist_id] += 1
    h5.close()
    
apply_to_all_files(msd_subset_path, func=func_to_count_artist_id)

10001

# FIND MOST POPULAR ARTIST

In [32]:
# most popular artist (with most songs) is
most_pop_aid = sorted(files_per_artist,
                      key=files_per_artist.__getitem__,
                      reverse=True)[0]
print most_pop_aid, 'has', files_per_artist[most_pop_aid], 'songs.'

AROIHOI122988FEB8E has 13 songs.


In [34]:
conn = sqlite3.connect(os.path.join(msd_subset_addf_path,
                                    'subset_track_metadata.db'))
q = "SELECT DISTINCT artist_name FROM songs"
q += " WHERE artist_id='" + most_pop_aid + "'"
res = conn.execute(q)
pop_artist_names = map(lambda x : x[0], res.fetchall())
conn.close()
print 'SQL query: ' + q

SQL query: SELECT DISTINCT artist_name FROM songs WHERE artist_id='AROIHOI122988FEB8E'


In [35]:
print "name(s) of most popular artists: " , pop_artist_names

name(s) of most popular artists:  [u'Mario Rosenstock']


In [38]:
# let's do all this work in SQLite in a few seconds
t1 = time.time()
conn = sqlite3.connect(os.path.join(msd_subset_addf_path,
                                    "subset_track_metadata.db"))
q = "SELECT DISTINCT artist_id,artist_name,Count(track_id) FROM songs"
q += " GROUP BY artist_id"
res = conn.execute(q)
pop_artists = res.fetchall()
conn.close()
t2 = time.time()
print "found most popular artist in " , pop_artist_names
print sorted(pop_artist_names, lambda x:x[2], reverse=True)[0]

found most popular artist in  [u'Mario Rosenstock']
Mario Rosenstock
